In [1]:
1+1

2

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
try:
    sys.path.remove("../py_src") # Hacky way to add higher directory to python modules path. 
except:
    pass
sys.path.append("../py_src") # Means that I dont have to make src/ a proper python package

In [4]:
from system_parameters import SystemParameters
from pulsars import Pulsars
from gravitational_waves import GWs
from synthetic_data import SyntheticData
from plotting import plot_statespace,plot_all
from model import LinearModel
from kalman_filter import KalmanFilter
from bilby_wrapper import BilbySampler
from priors import priors_dict,bilby_priors_dict

In [48]:
P   = SystemParameters()       # define the system parameters as a class
PTA = Pulsars(P)               # setup the PTA
GW  = GWs(P)                   # setup GW related constants and functions. This is a dict, not a class, for interaction later with Bilby 
data = SyntheticData(PTA,GW,1) # generate some synthetic data


#Define the model 
model = LinearModel

#Initialise the Kalman filter
KF = KalmanFilter(model,data.f_measured,PTA)

true_parameters = priors_dict(PTA,GW)



def likelihood_wrapper(p):

    g = true_parameters.copy()
    g["omega_gw"] = p
    #print("LETS GOOOOO")
    model_likelihood,model_predictions = KF.likelihood_and_states(g)

    #plot_all(PTA.t,data.intrinsic_frequency,data.f_measured,model_predictions,psr_index =0)
    #print(model_likelihood)

    #print(-model_likelihood)
    return -model_likelihood


from scipy.optimize import minimize


import numpy as np 
x0 = np.array([5.001e-7])


res = minimize(likelihood_wrapper, x0, method='BFGS', tol=1e-12)
value = res.x[0]
l = likelihood_wrapper(value)


print(value)
print(l)


g = true_parameters.copy()
g["omega_gw"] = 5e-7
model_likelihood,model_predictions = KF.likelihood_and_states(g)
print(-model_likelihood)


The number of pulsars is:  47
5.001e-07
11817980932655.445
22545.238888357337


In [35]:
value 

9.999022554373368e-06

In [24]:
x0.dtype

dtype('float64')